Modificaciones pendientes por agregar al código principal

Filtro para columna condicional

In [ ]:
# Filtro para verificar 
filtro = merge

In [ ]:
# Estos son los filtros para crear la nueva columna que servira para filtrar la informacion
filtro = merge[merge['Largo_x'] != merge['Largo_y']]
#862
filtro = merge[merge['Número de Vehículo'] != merge['Número de secuencia de carro']]
#12200  
filtro = merge[merge['Secuencia de arcos_x'] != merge['Secuencia de arcos_y']]
#replece blanco y negro a byn
merge['Concesionario Programado'].replace('Blanco y Negro Masivo', 'BYN', inplace=True)

In [ ]:
# Condiciones para la nueva columna
condicion1 = merge['Largo_x'] != merge['Largo_y']
condicion2 = merge['Número de Vehículo'] != merge['Número de secuencia de carro']
condicion3 = merge['Secuencia de arcos_x'] != merge['Secuencia de arcos_y']
condicion4 = merge['Concesionario de Transporte_x'] != merge['Concesionario Programado']

# Aplicar condiciones y asignar valores a la nueva columna
filtro['condicional'] = np.where(condicion1 & condicion2 & condicion3, 'Revisar largo, parada y vehículo',
                                 np.where(condicion1, 'Revisar largo', 
                                          np.where(condicion2, 'Revisar numero de vehiculo',
                                                   np.where(condicion3, 'Revisar paradas',
                                                            np.where(condicion4, 'Revisar consesionario', 'OK')))))


In [ ]:
filtro['condicional'].value_counts()

Incumplido

In [ ]:
# Aplicar la lógica a cada fila del DataFrame y asignar el resultado a una nueva columna
merge['resultado'] = merge.apply(lambda row: \
    2 if (row['Tipo Kilómetros'] == "Kms en Servicio" and pd.isna(row['AC193'])) \
      and row['Tipo de viaje corto'] == "Viaje comercial" \
    else (1 if (row['Tipo Kilómetros'] == "Kms en Servicio" and pd.isna(row['AC193'])) \
      and ((row['Tipo de viaje corto'] == "Viaje de punto operacional" or row['Tipo de viaje corto'] == "Viaje de posicionamiento") \
      and (pd.isna(row['Número de Vehículo']) and (row['AH194'] == 2 and (row['AF193'] == 0 or row['AF193'] == 1) \
      or ((row['AF193'] != 0 and row['AF193'] != 1) or (row['AF194'] == 0 or row['AF194'] == 1)) and (pd.notna(row['AH195']) or 2))))) \
    else (0 if (row['AC193'] == "CAMBIO" or row['AC193'] == "CANCELADA") and row['G193'] != "Kms en Vacio" else ""), axis=1)

In [ ]:
if G193 != "Kms en Vacio":
    if P193 != AF193:
        if AD193 == "":
            resultado = "#Error"
        else:
            resultado = "Arco va desde " + AD193 + " hasta " + AE193 + " con un arco de " + str(AF193) + ", revisar calificacion"
    elif W193 != AH193:
        resultado = "Calificacion debe ser " + str(AH193) + " verificar"
    elif H193 and H193[1] != AC193[1]:
        if AC193 == "" or AC193 == 0:
            if H193 == "":
                resultado = ""
            else:
                resultado = "Verificar Vehiculo: " + H193
        else:
            resultado = "Actualizar vehiculo: " + str(AC193)
    else:
        resultado = ""
else:
    resultado = ""

Jerarquias, este codigo toma la info de descriciones y crea dos columnas causales y novedades

In [ ]:
prueba = bitacora['Jerarquía'].drop_duplicates().to_frame()

In [ ]:
# Función para dividir el texto en base a ":" y seleccionar la parte deseada
def split_text(text):
    if ":" in text:
        # Dividir el texto en base a ":" y tomar la primera parte
        return text.split(":", 1)[0]
    else:
        return text

In [ ]:
# Aplicar la función a la columna 'Jerarquía'
prueba['Texto_Antes'] = prueba['Jerarquía'].apply(split_text)

In [ ]:
# Funcion para obtener el ultimo dato de la columna
def obtener_texto_despues_ultimo(texto):
    # Usamos una expresión regular para encontrar el texto después del último ":"
    match = re.search(r'[^:]*$', texto)
    if match:
        return match.group(0)
    else:
        return ''

In [ ]:

prueba['despues_text'] = prueba['Jerarquía'].apply(obtener_texto_despues_ultimo)


In [ ]:
# Se cambian los vehiculos del cierre por los de k2##
mergebitacora['Número de secuencia de carro'] = [x if pd.notna(x) else y for x, y in zip(mergebitacora['Número de secuencia de carro'], mergebitacora['Número de Vehículo'])]
# Marca en la columnas Cambios los cambios que realiza el código
mergebitacora['condicion'] = [ np.nan if  pd.notna(x) else 'Se Cambia # V RF' for x, y in zip(mergebitacora['Número de secuencia de carro'], mergebitacora['Número de Vehículo'])]
# Función para actualizar los valores de Largo_y si Largo_x es mayor
def actualizar_largo(row):
    if pd.isna(row['Largo_y']):
        return row['Largo_x']
    elif row['Largo_x'] > row['Largo_y']:
        return row['Largo_x']
    else:
        return row['Largo_y']
# Función para actualizar los valores de Largo_y si Largo_x es mayor
def actualizar_largo_texto(row):
    if pd.isna(row['Largo_y']) and pd.isna(row['condicion']) :
        return 'Cambia por NAN'
    elif pd.isna(row['Largo_y']) and pd.notnull(row['condicion']) :
        return row['condicion'] + '/Cambia por NAN'
    elif row['Largo_x'] > row['Largo_y'] and pd.isna(row['condicion']):
        return 'Cambia x Mayor1'
    elif row['Largo_x'] > row['Largo_y'] and pd.notnull(row['condicion']) :
        return row['condicion'] + '/Cambia x Mayor2'
    else:
        if pd.notnull(row['condicion']):
            return row['condicion']
# Aplicar la función a la columna Largo_y
mergebitacora['Largo_y'] = mergebitacora.apply(actualizar_largo, axis=1)

# Marcar las filas donde se realizan cambios en una nueva columna
mergebitacora['condicion'] = mergebitacora.apply(actualizar_largo_texto, axis=1)